In [22]:
include("../src/ColorfulFCM.jl")

Main.ColorfulFCM

$$ \frac{\alpha_{ex}}{100} \cdot \langle \alpha_{em} \rangle \cdot \alpha_L$$

In [23]:
Device = ColorfulFCM.ThermoFisherDevice

marker_file = "../data/使用抗体.csv"
emission_file = "../data/常见染料_发射曲线.csv"
excitation_file = "../data/常见染料_激发曲线.csv"

readin_marker_density = [
    (:B220,  10000), 
    (:IgD,   4000), 
    (:HEL,   4000),
    (:GL_7,  10000),
    (:FAS,   3000),
    (:CD138, 3000),
    (:CD45_1,3000),
    (:DEAD,  30000),
    #(:CD23, 10000),
]

interaction = ColorfulFCM.get_interaction_sheet(readin_marker_density, marker_file, emission_file, excitation_file, Device)
@time solution = ColorfulFCM.solve(interaction, readin_marker_density);

 10.141154 seconds (45.36 M allocations: 6.218 GiB, 14.32% gc time)


In [24]:
(value, optimal_idx) = [sum(item.score) for item in solution] |> findmax
solution[optimal_idx]

,laser,lens_mu,lens_sig,marker,dye,score
,Symbol,Int64,Int64,Symbol,Symbol,Float64
1,red,660,20,FAS,APC,18428.4
2,red,730,45,HEL,Alexa647,6675.3
3,blue,575,26,GL_7,PE,10308.7
4,violet,450,50,CD138,BV421,10324.5
5,blue,710,50,CD45_1,Percp_Cy5_5,18810.3
6,violet,605,40,B220,BV605,21132.7
7,blue,530,30,IgD,FITC,18210.2
8,red,780,60,DEAD,eFluor780,2255.22
